In [9]:
import pandas as pd
from pulp import *

# Import data
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col=0)
freight_costs = pd.read_excel('freight_costs.xlsx', index_col=0)
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col=0)
cap = pd.read_excel('capacity.xlsx', index_col=0)
demand = pd.read_excel('demand.xlsx', skiprows=1, usecols=[1, 2], names=['Location', 'Demand'])

# Clean and prepare data
manvar_costs.index = manvar_costs.index.astype(str).str.strip()
freight_costs.index = freight_costs.index.astype(str).str.strip()
fixed_costs.index = fixed_costs.index.astype(str).str.strip()
cap.index = cap.index.astype(str).str.strip()
demand['Location'] = demand['Location'].astype(str).str.strip()
demand = demand.set_index('Location')

# Calculate variable costs
var_cost = freight_costs / 1000 + manvar_costs

# Sensitivity Analysis Function
def run_sensitivity_analysis(demand_multiplier=1.0, cost_multiplier=1.0):
    # Define decision variables and model
    loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
    size = ['Low', 'High']
    model = LpProblem("Capacitated Plant Location Model - Sensitivity Analysis", LpMinimize)
    x = LpVariable.dicts("production_", [(i, j) for i in loc for j in loc], lowBound=0, upBound=None, cat='Continuous')
    y = LpVariable.dicts("plant_", [(i, s) for s in size for i in loc], cat='Binary')

    # Adjust demand and costs for sensitivity analysis
    adjusted_demand = {k: v * demand_multiplier for k, v in demand['Demand'].items()}
    adjusted_fixed_costs = fixed_costs * cost_multiplier
    adjusted_var_cost = var_cost * cost_multiplier

    # Define objective function with adjusted costs
    model += (lpSum([adjusted_fixed_costs.loc[i, s] * y[(i, s)] * 1000 for s in size for i in loc])
              + lpSum([adjusted_var_cost.loc[i, j] * x[(i, j)] for i in loc for j in loc]))

    # Add constraints with adjusted demand
    for j in loc:
        if j in adjusted_demand:
            model += lpSum([x[(i, j)] for i in loc]) == adjusted_demand[j]
    for i in loc:
        model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i, s] * y[(i, s)] * 1000 for s in size])

    # Solve model
    model.solve()

    # Return the results
    total_costs = int(value(model.objective))
    status = LpStatus[model.status]
    return total_costs, status

# Example Sensitivity Analysis Scenarios
scenarios = [(1.0, 1.0), (1.1, 1.0), (1.0, 1.1), (0.9, 1.0), (1.0, 0.9)]
for demand_mult, cost_mult in scenarios:
    total_costs, status = run_sensitivity_analysis(demand_multiplier=demand_mult, cost_multiplier=cost_mult)
    print(f"Scenario with Demand Multiplier={demand_mult}, Cost Multiplier={cost_mult}: Total Costs = {total_costs} USD, Status = {status}")


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Scenario with Demand Multiplier=1.0, Cost Multiplier=1.0: Total Costs = 92981000 USD, Status = Optimal
Scenario with Demand Multiplier=1.1, Cost Multiplier=1.0: Total Costs = 101940100 USD, Status = Optimal
Scenario with Demand Multiplier=1.0, Cost Multiplier=1.1: Total Costs = 102279100 USD, Status = Optimal
Scenario with Demand Multiplier=0.9, Cost Multiplier=1.0: Total Costs = 84021900 USD, Status = Optimal
Scenario with Demand Multiplier=1.0, Cost Multiplier=0.9: Total Costs = 83682900 USD, Status = Optimal
